In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
from sklearn import model_selection
from sklearn.neural_network import MLPClassifier
import pickle 
import json

In [2]:
df = pd.read_json("uber.json")

In [3]:
X = df['Statement']
Y = df['Label']

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer()
vect.fit(X)
X = vect.transform(X)

In [5]:
from scipy.sparse import coo_matrix
X_sparse = coo_matrix(X)
from sklearn.utils import shuffle
X, X_sparse, Y = shuffle(X, X_sparse, Y, random_state=0)

In [6]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=10)
kf.get_n_splits(X)
print(kf)

KFold(n_splits=10, random_state=None, shuffle=False)


In [7]:
for train_index, test_index in kf.split(X):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]

In [12]:
model = MLPClassifier(hidden_layer_sizes=(13,13,13),max_iter=500)

In [13]:
model.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(13, 13, 13), learning_rate='constant',
       learning_rate_init=0.001, max_iter=500, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [14]:
filename = 'NN.sav'

In [15]:
pickle.dump(model, open(filename, 'wb'))

In [16]:
loaded_model = pickle.load(open(filename, 'rb'))

In [17]:
result = loaded_model.score(X_test, y_test)

In [18]:
print(result)

0.8134328358208955


In [19]:
neural = MLPClassifier()
%time neural.fit(X_train, y_train)
y_pred_class = neural.predict(X_test)
y_pred_prob = neural.predict_proba(X_test)[:, 1]

Wall time: 1min 6s


In [20]:
print('Confusion matrix:\n',confusion_matrix(y_test,y_pred_class),'\n')
print('Classification report:\n',classification_report(y_test,y_pred_class))

Confusion matrix:
 [[109   3]
 [ 19   3]] 

Classification report:
               precision    recall  f1-score   support

           0       0.85      0.97      0.91       112
           1       0.50      0.14      0.21        22

   micro avg       0.84      0.84      0.84       134
   macro avg       0.68      0.55      0.56       134
weighted avg       0.79      0.84      0.79       134

